In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option("display.max_columns", None)

# Ricostruire raccordo CP2021 con ISCO livello 4
- [2025-03-26] F. Trentini ci ha indicato il database scaricabile da qui:https://esco.ec.europa.eu/en/use-esco/download con il raccordo in oggetto.
Si scarica un file .zip che contiene una serie di csv

## Transcodifica dei codici URI delle professioni

In [2]:
# Read the CSV starting from the header line

# OCCUPATIONS
occupations = (pd.read_csv("./data/classificazioni/ESCO dataset - v1.1.1 - classification - en - csv/occupations_en.csv", dtype=str)
               [['conceptUri', 'code', 'preferredLabel', 'iscoGroup']]
               .rename(columns={'conceptUri': 'occupationUri'
                                , 'code': 'occupation_code'
                                , 'preferredLabel': 'occupation_descr'
                                , 'iscoGroup': 'occupation4d_id'})
              )
occupations['occupation4d_id'] = 'OC'+occupations['occupation4d_id']
occupations.to_parquet("./data/classificazioni/esco_occupations_isco_4.parquet")


# Skill (prendo questo file perché ha già il codice usato nel DB WIH)
skills = (pd.read_excel("./data/codelist_escoskill_v111_v1.xlsx")
          [['skill_id', 'esco_v0101_uri', 'esco_v0101_description']]
          .rename(columns={'esco_v0101_uri': 'skillUri'
                           , 'esco_v0101_description': 'skill_descr'})
         )


# RELATIONS OCCUPATIONS-SKILLS (skill per professione decise a tavolino)
occupation_skill_relations = (pd.read_csv("./data/classificazioni/ESCO dataset - v1.1.1 - classification - en - csv/occupationSkillRelations_en.csv", dtype=str)
                              .merge(occupations, on='occupationUri', how='inner') # Si perdono 67 combinazioni su oltre 23.000
                              .merge(skills, on='skillUri', how='inner') # Si duplicano delle righe perché per alcuni URI abbiamo più skill_id --> Va bene!
                             )[['occupationUri', 'occupation_code', 'occupation_descr', 'occupation4d_id'
                                , 'relationType'
                                , 'skillUri'
                                , 'skill_id'
                                , 'skill_descr'
                               ]]
occupation_skill_relations.to_parquet("./data/classificazioni/esco_occupation_skill_relations.parquet")



In [4]:
occupations.query("occupation_descr.str.contains('database')", engine='python').head()

,occupationUri,occupation_code,occupation_descr,occupation4d_id
759,http://data.europa.eu/esco/occupation/3d190639...,2521.4,database integrator,OC2521
1724,http://data.europa.eu/esco/occupation/8c57af09...,2521.1,database administrator,OC2521
1742,http://data.europa.eu/esco/occupation/8d9ec84d...,2521.2,database designer,OC2521
2138,http://data.europa.eu/esco/occupation/b11e1742...,2521.3,database developer,OC2521


## Creazione raccordo CP2021 - ISCO 4 digit

In [8]:
map_transcode = (pd.read_csv("./data/classificazioni/EURESmapping_occs_IT.csv", skiprows=16)
                 .rename(columns={'Classification 1 URI': 'occupationUri'
                                  , 'Classification 1 PrefLabel': 'occupation_desc_it'
                                  , 'Classification 2 ID': 'cp2021'
                                  , 'Classification 2 PrefLabel': 'cp2021_desc'
                                 })
                 [['occupationUri', 'occupation_desc_it', 'cp2021', 'cp2021_desc']]
                 .merge(occupations, how='inner', on='occupationUri')
                )


raccordo_istat_isco4 = (map_transcode[['cp2021', 'cp2021_desc', 'occupation4d_id', 'occupation_desc_it']].drop_duplicates()
                        .sort_values(by=['cp2021', 'occupation4d_id'])
                        .query("cp2021.isnull()==False and occupation4d_id.isnull()==False", engine='python')
                       )
raccordo_istat_isco4.to_parquet("./data/classificazioni/esco_raccordo_istat_isco4.parquet")

In [10]:
raccordo_istat_isco4.to_clipboard()